In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12334,2024-11-20,Eua Nba,00:30,Los Angeles Lakers,Utah Jazz,1.18,5.28,234.5,1.84,2.00,-11.5,2.02,1.83,lY0m2Dht,0.847458,0.189394,0.543478,0.500000,0.036852,0.6,0.4,NaN,NaN,171.418,40.384217,0.235589,372.428,73.756139,0.198041,184.394,442.212,172.80,351.87,0.0,0.0,0.0,0.0,0.897566,0.058926,0.069792,2.40,0.480,0.375000,0.584338,0.7,0.115662,1.24,0.248,17.258065,0.358839,0.3,-0.058839
12335,2024-11-20,Eua Nba,21:30,Cleveland Cavaliers,New Orleans Pelicans,1.11,7.38,220.5,1.83,1.98,-14.5,2.01,1.80,zVMpQa8C,0.900901,0.135501,0.546448,0.505051,0.036402,0.8,0.4,NaN,NaN,187.150,74.598790,0.398604,364.944,167.007072,0.457624,167.966,326.670,120.75,469.56,0.0,0.0,0.0,0.0,1.044419,0.055678,0.077949,1.81,0.362,0.303867,0.654945,0.6,-0.054945,-5.00,-1.000,-6.380000,0.473281,0.1,-0.373281
12336,2024-11-20,Eua Nba,21:30,Milwaukee Bucks,Chicago Bulls,1.32,3.60,238.5,1.93,1.93,-9.5,2.05,1.77,WdGgOwwP,0.757576,0.277778,0.518135,0.518135,0.035354,0.4,0.8,NaN,NaN,228.018,111.615186,0.489502,403.212,85.250099,0.211427,158.412,378.132,184.15,448.88,0.0,0.0,0.0,0.0,0.655367,0.000000,0.103660,-1.11,-0.222,-1.441441,0.647729,0.6,-0.047729,4.27,0.854,3.044496,0.364096,0.5,0.135904
12337,2024-11-20,Eua Nba,22:00,Houston Rockets,Indiana Pacers,1.43,2.95,230.5,1.93,1.93,-6.5,1.96,1.90,Khe3rsOt,0.699301,0.338983,0.518135,0.518135,0.038284,0.6,0.6,NaN,NaN,215.434,140.493640,0.652142,215.426,97.285074,0.451594,172.392,232.626,167.61,158.40,0.0,0.0,0.0,0.0,0.490777,0.000000,0.021983,1.18,0.236,1.822034,0.666600,0.7,0.033400,-2.40,-0.480,-4.062500,0.488285,0.3,-0.188285
12338,2024-11-20,Eua Nba,22:00,Oklahoma City Thunder,Portland Trail Blazers,1.09,8.50,223.5,1.93,1.93,-14.5,1.98,1.82,lYnKvaN5,0.917431,0.117647,0.518135,0.518135,0.035078,0.6,0.4,NaN,NaN,147.128,25.823883,0.175520,420.140,200.104469,0.476280,142.320,450.652,113.42,667.08,0.0,1.0,0.0,0.0,1.092734,0.000000,0.059546,-0.22,-0.044,-2.045455,0.785503,0.7,-0.085503,0.91,0.182,41.208791,0.341161,0.4,0.058839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12400,2024-11-20,Finlândia Korisliiga Feminina,13:30,HBA-Marsky Helsinki F,Forssan Alku F,1.21,3.74,160.5,1.83,1.83,-10.5,1.95,1.72,8tKst3mn,0.826446,0.267380,0.546448,0.546448,0.093826,0.4,0.6,NaN,NaN,308.494,181.379644,0.587952,239.288,219.832709,0.918695,327.576,226.028,206.55,630.00,0.0,1.0,0.0,0.0,0.722820,0.000000,0.088629,-2.84,-0.568,-0.369718,0.000000,0.0,0.000000,-1.67,-0.334,-8.203593,0.000000,0.0,0.000000
12401,2024-11-20,Islândia Premier League Feminina,16:15,Keflavik F,Tindastoll F,1.28,3.24,156.5,1.85,1.81,-10.5,1.90,1.77,0jJ29275,0.781250,0.308642,0.540541,0.552486,0.089892,0.8,0.0,NaN,NaN,116.418,23.604689,0.202758,NaN,NaN,NaN,113.380,0.000,128.04,204.25,0.0,0.0,0.0,0.0,0.613243,0.015456,0.050095,1.30,0.260,1.076923,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12402,2024-11-20,Islândia Premier League Feminina,16:15,Thor AK Akureyri F,Athena F,1.29,3.15,164.5,1.88,1.78,-8.5,1.93,1.74,YNagDp8t,0.775194,0.317460,0.531915,0.561798,0.092654,1.0,0.0,NaN,NaN,170.160,76.171406,0.447646,194.742,103.541761,0.531687,149.038,0.000,162.18,221.76,0.0,0.0,0.0,0.0,0.592441,0.038640,0.073215,0.56,0.112,2.589286,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12403,2024-11-20,Islândia Premier L

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:30,Eua Nba,Los Angeles Lakers,Utah Jazz,234.5,1.84,0.8855,Over
1,22:00,Eua Nba,Houston Rockets,Indiana Pacers,230.5,1.93,0.9668,Over
2,22:00,Eua Nba,Memphis Grizzlies,Philadelphia 76ers,218.5,1.84,0.9603,Over
